# Prediction

**Paper:** Automatic identification of Hainan Gibbon calls in passive acoustic recordings

**Authors:** Emmanuel Dufourq, Ian Durbach, James Hansford, Sam Turvey, Amanda Hoepfner

**Year:** March 2020

**Repository:** https://github.com/emmanueldufourq/GibbonClassifier

Predict on a single .wav audio file.

The weights are saved in the '/Predictions' folder, and the name of the weights file to use is specified in the 'weights_name' variable. In this example the weights file is 'pretrained_weights.hdf5'. The testing file in this example is 'HGSM3B_0+1_20160308_055700.wav' and the location of the test files are in '/Raw_Data/Test/'.

The same time segments which were used in training should be used when testing the model. For example, if 10 second segments were used in the 'Extract_Audio' notebook, then the model expects 10 second inputs for prediction. This is saved in the variable 'time_to_extract'. The correct sampling rate should be used (same value as was used in training). 

Two output files are produced in the folder '/Preedictions/'. The one end with 'binary_prediction.txt' and the other 'prediction.txt'. The former contains the binary predictions as either 0 (non-gibbon) or 1 (gibbon). The latter contains two values (softmax probabilistic output). The values are [probability non-gibbon, probability gibbon]. In this example the output files are named 'HGSM3B_0+1_20160308_055700.wav_binary_prediction.txt' and 'HGSM3B_0+1_20160308_055700.wav_prediction.txt'.

In [1]:
import pandas as pd
import soundfile as sf
from os import listdir
import librosa
import collections

from Augmentation import convert_to_image
from CNN_Network import *
from Predict_Helper import *

Using TensorFlow backend.


## Parameters

In [2]:
testing_file = 'HGSM3B_0+1_20160308_055700.wav'
testing_folder = '../Raw_Data/Test/'
prediction_folder = '../Predictions/'
weights_name = 'pretrained_weights.hdf5'
location_model = "../Experiments/"
output_directory = ''
time_to_extract = 10
sample_rate = 4800

In [3]:
test_file_audio, test_file_sample_rate = librosa.load(testing_folder + testing_file, 
                                                      sr=sample_rate) 

## Check the sample rate from the file

In [4]:
test_file_sample_rate

4800

## Extract segments from test file

In [5]:
X = create_X_new(test_file_audio, time_to_extract, 
                 sample_rate, verbose = 1)

-----------------------
start (seconds) 0
end (seconds) 28800.0
duration (seconds) 28800.0

Chunks: 28790
Index: 0
Chunk start time (sec): 0
Chunk end time (sec): 10
Index: 1
Chunk start time (sec): 1
Chunk end time (sec): 11
Index: 2
Chunk start time (sec): 2
Chunk end time (sec): 12
Index: 3
Chunk start time (sec): 3
Chunk end time (sec): 13
Index: 4
Chunk start time (sec): 4
Chunk end time (sec): 14
Index: 5
Chunk start time (sec): 5
Chunk end time (sec): 15
Index: 6
Chunk start time (sec): 6
Chunk end time (sec): 16
Index: 7
Chunk start time (sec): 7
Chunk end time (sec): 17
Index: 8
Chunk start time (sec): 8
Chunk end time (sec): 18
Index: 9
Chunk start time (sec): 9
Chunk end time (sec): 19
Index: 10
Chunk start time (sec): 10
Chunk end time (sec): 20
Index: 11
Chunk start time (sec): 11
Chunk end time (sec): 21
Index: 12
Chunk start time (sec): 12
Chunk end time (sec): 22
Index: 13
Chunk start time (sec): 13
Chunk end time (sec): 23
Index: 14
Chunk start time (sec): 14
Chunk end

Index: 504
Chunk start time (sec): 504
Chunk end time (sec): 514
Index: 505
Chunk start time (sec): 505
Chunk end time (sec): 515
Index: 506
Chunk start time (sec): 506
Chunk end time (sec): 516
Index: 507
Chunk start time (sec): 507
Chunk end time (sec): 517
Index: 508
Chunk start time (sec): 508
Chunk end time (sec): 518
Index: 509
Chunk start time (sec): 509
Chunk end time (sec): 519
Index: 510
Chunk start time (sec): 510
Chunk end time (sec): 520
Index: 511
Chunk start time (sec): 511
Chunk end time (sec): 521
Index: 512
Chunk start time (sec): 512
Chunk end time (sec): 522
Index: 513
Chunk start time (sec): 513
Chunk end time (sec): 523
Index: 514
Chunk start time (sec): 514
Chunk end time (sec): 524
Index: 515
Chunk start time (sec): 515
Chunk end time (sec): 525
Index: 516
Chunk start time (sec): 516
Chunk end time (sec): 526
Index: 517
Chunk start time (sec): 517
Chunk end time (sec): 527
Index: 518
Chunk start time (sec): 518
Chunk end time (sec): 528
Index: 519
Chunk start ti

Chunk start time (sec): 1018
Chunk end time (sec): 1028
Index: 1019
Chunk start time (sec): 1019
Chunk end time (sec): 1029
Index: 1020
Chunk start time (sec): 1020
Chunk end time (sec): 1030
Index: 1021
Chunk start time (sec): 1021
Chunk end time (sec): 1031
Index: 1022
Chunk start time (sec): 1022
Chunk end time (sec): 1032
Index: 1023
Chunk start time (sec): 1023
Chunk end time (sec): 1033
Index: 1024
Chunk start time (sec): 1024
Chunk end time (sec): 1034
Index: 1025
Chunk start time (sec): 1025
Chunk end time (sec): 1035
Index: 1026
Chunk start time (sec): 1026
Chunk end time (sec): 1036
Index: 1027
Chunk start time (sec): 1027
Chunk end time (sec): 1037
Index: 1028
Chunk start time (sec): 1028
Chunk end time (sec): 1038
Index: 1029
Chunk start time (sec): 1029
Chunk end time (sec): 1039
Index: 1030
Chunk start time (sec): 1030
Chunk end time (sec): 1040
Index: 1031
Chunk start time (sec): 1031
Chunk end time (sec): 1041
Index: 1032
Chunk start time (sec): 1032
Chunk end time (sec

Chunk start time (sec): 1357
Chunk end time (sec): 1367
Index: 1358
Chunk start time (sec): 1358
Chunk end time (sec): 1368
Index: 1359
Chunk start time (sec): 1359
Chunk end time (sec): 1369
Index: 1360
Chunk start time (sec): 1360
Chunk end time (sec): 1370
Index: 1361
Chunk start time (sec): 1361
Chunk end time (sec): 1371
Index: 1362
Chunk start time (sec): 1362
Chunk end time (sec): 1372
Index: 1363
Chunk start time (sec): 1363
Chunk end time (sec): 1373
Index: 1364
Chunk start time (sec): 1364
Chunk end time (sec): 1374
Index: 1365
Chunk start time (sec): 1365
Chunk end time (sec): 1375
Index: 1366
Chunk start time (sec): 1366
Chunk end time (sec): 1376
Index: 1367
Chunk start time (sec): 1367
Chunk end time (sec): 1377
Index: 1368
Chunk start time (sec): 1368
Chunk end time (sec): 1378
Index: 1369
Chunk start time (sec): 1369
Chunk end time (sec): 1379
Index: 1370
Chunk start time (sec): 1370
Chunk end time (sec): 1380
Index: 1371
Chunk start time (sec): 1371
Chunk end time (sec

Index: 1933
Chunk start time (sec): 1933
Chunk end time (sec): 1943
Index: 1934
Chunk start time (sec): 1934
Chunk end time (sec): 1944
Index: 1935
Chunk start time (sec): 1935
Chunk end time (sec): 1945
Index: 1936
Chunk start time (sec): 1936
Chunk end time (sec): 1946
Index: 1937
Chunk start time (sec): 1937
Chunk end time (sec): 1947
Index: 1938
Chunk start time (sec): 1938
Chunk end time (sec): 1948
Index: 1939
Chunk start time (sec): 1939
Chunk end time (sec): 1949
Index: 1940
Chunk start time (sec): 1940
Chunk end time (sec): 1950
Index: 1941
Chunk start time (sec): 1941
Chunk end time (sec): 1951
Index: 1942
Chunk start time (sec): 1942
Chunk end time (sec): 1952
Index: 1943
Chunk start time (sec): 1943
Chunk end time (sec): 1953
Index: 1944
Chunk start time (sec): 1944
Chunk end time (sec): 1954
Index: 1945
Chunk start time (sec): 1945
Chunk end time (sec): 1955
Index: 1946
Chunk start time (sec): 1946
Chunk end time (sec): 1956
Index: 1947
Chunk start time (sec): 1947
Chunk e

...

Index: 26427
Chunk start time (sec): 26427
Chunk end time (sec): 26437
Index: 26428
Chunk start time (sec): 26428
Chunk end time (sec): 26438
Index: 26429
Chunk start time (sec): 26429
Chunk end time (sec): 26439
Index: 26430
Chunk start time (sec): 26430
Chunk end time (sec): 26440
Index: 26431
Chunk start time (sec): 26431
Chunk end time (sec): 26441
Index: 26432
Chunk start time (sec): 26432
Chunk end time (sec): 26442
Index: 26433
Chunk start time (sec): 26433
Chunk end time (sec): 26443
Index: 26434
Chunk start time (sec): 26434
Chunk end time (sec): 26444
Index: 26435
Chunk start time (sec): 26435
Chunk end time (sec): 26445
Index: 26436
Chunk start time (sec): 26436
Chunk end time (sec): 26446
Index: 26437
Chunk start time (sec): 26437
Chunk end time (sec): 26447
Index: 26438
Chunk start time (sec): 26438
Chunk end time (sec): 26448
Index: 26439
Chunk start time (sec): 26439
Chunk end time (sec): 26449
Index: 26440
Chunk start time (sec): 26440
Chunk end time (sec): 26450
Index:

Chunk start time (sec): 27050
Chunk end time (sec): 27060
Index: 27051
Chunk start time (sec): 27051
Chunk end time (sec): 27061
Index: 27052
Chunk start time (sec): 27052
Chunk end time (sec): 27062
Index: 27053
Chunk start time (sec): 27053
Chunk end time (sec): 27063
Index: 27054
Chunk start time (sec): 27054
Chunk end time (sec): 27064
Index: 27055
Chunk start time (sec): 27055
Chunk end time (sec): 27065
Index: 27056
Chunk start time (sec): 27056
Chunk end time (sec): 27066
Index: 27057
Chunk start time (sec): 27057
Chunk end time (sec): 27067
Index: 27058
Chunk start time (sec): 27058
Chunk end time (sec): 27068
Index: 27059
Chunk start time (sec): 27059
Chunk end time (sec): 27069
Index: 27060
Chunk start time (sec): 27060
Chunk end time (sec): 27070
Index: 27061
Chunk start time (sec): 27061
Chunk end time (sec): 27071
Index: 27062
Chunk start time (sec): 27062
Chunk end time (sec): 27072
Index: 27063
Chunk start time (sec): 27063
Chunk end time (sec): 27073
Index: 27064
Chunk 

Chunk end time (sec): 27688
Index: 27679
Chunk start time (sec): 27679
Chunk end time (sec): 27689
Index: 27680
Chunk start time (sec): 27680
Chunk end time (sec): 27690
Index: 27681
Chunk start time (sec): 27681
Chunk end time (sec): 27691
Index: 27682
Chunk start time (sec): 27682
Chunk end time (sec): 27692
Index: 27683
Chunk start time (sec): 27683
Chunk end time (sec): 27693
Index: 27684
Chunk start time (sec): 27684
Chunk end time (sec): 27694
Index: 27685
Chunk start time (sec): 27685
Chunk end time (sec): 27695
Index: 27686
Chunk start time (sec): 27686
Chunk end time (sec): 27696
Index: 27687
Chunk start time (sec): 27687
Chunk end time (sec): 27697
Index: 27688
Chunk start time (sec): 27688
Chunk end time (sec): 27698
Index: 27689
Chunk start time (sec): 27689
Chunk end time (sec): 27699
Index: 27690
Chunk start time (sec): 27690
Chunk end time (sec): 27700
Index: 27691
Chunk start time (sec): 27691
Chunk end time (sec): 27701
Index: 27692
Chunk start time (sec): 27692
Chunk 

Chunk start time (sec): 28223
Chunk end time (sec): 28233
Index: 28224
Chunk start time (sec): 28224
Chunk end time (sec): 28234
Index: 28225
Chunk start time (sec): 28225
Chunk end time (sec): 28235
Index: 28226
Chunk start time (sec): 28226
Chunk end time (sec): 28236
Index: 28227
Chunk start time (sec): 28227
Chunk end time (sec): 28237
Index: 28228
Chunk start time (sec): 28228
Chunk end time (sec): 28238
Index: 28229
Chunk start time (sec): 28229
Chunk end time (sec): 28239
Index: 28230
Chunk start time (sec): 28230
Chunk end time (sec): 28240
Index: 28231
Chunk start time (sec): 28231
Chunk end time (sec): 28241
Index: 28232
Chunk start time (sec): 28232
Chunk end time (sec): 28242
Index: 28233
Chunk start time (sec): 28233
Chunk end time (sec): 28243
Index: 28234
Chunk start time (sec): 28234
Chunk end time (sec): 28244
Index: 28235
Chunk start time (sec): 28235
Chunk end time (sec): 28245
Index: 28236
Chunk start time (sec): 28236
Chunk end time (sec): 28246
Index: 28237
Chunk 

Index: 28738
Chunk start time (sec): 28738
Chunk end time (sec): 28748
Index: 28739
Chunk start time (sec): 28739
Chunk end time (sec): 28749
Index: 28740
Chunk start time (sec): 28740
Chunk end time (sec): 28750
Index: 28741
Chunk start time (sec): 28741
Chunk end time (sec): 28751
Index: 28742
Chunk start time (sec): 28742
Chunk end time (sec): 28752
Index: 28743
Chunk start time (sec): 28743
Chunk end time (sec): 28753
Index: 28744
Chunk start time (sec): 28744
Chunk end time (sec): 28754
Index: 28745
Chunk start time (sec): 28745
Chunk end time (sec): 28755
Index: 28746
Chunk start time (sec): 28746
Chunk end time (sec): 28756
Index: 28747
Chunk start time (sec): 28747
Chunk end time (sec): 28757
Index: 28748
Chunk start time (sec): 28748
Chunk end time (sec): 28758
Index: 28749
Chunk start time (sec): 28749
Chunk end time (sec): 28759
Index: 28750
Chunk start time (sec): 28750
Chunk end time (sec): 28760
Index: 28751
Chunk start time (sec): 28751
Chunk end time (sec): 28761
Index:

In [6]:
X = convert_to_image(X)

In [7]:
X.shape

(28790, 128, 188, 1)

## Build the model and load weights

In [8]:
model = network()
model.load_weights(location_model+weights_name)

## Predict

In [9]:
model_prediction = model.predict(X, batch_size=128)
binary_prediction = np.argmax(model_prediction,1)

## Save predictions to file

In [10]:
np.savetxt(prediction_folder + testing_file + '_prediction.txt',model_prediction, fmt='%5f')
np.savetxt(prediction_folder + testing_file + '_binary_prediction.txt',binary_prediction, fmt='%i')